# Session 5

1. Combiner les données
2. Stratégie split-apply-combine (groupby)
3. Index hiérarchiques
4. Méthodes de reshaping (2)

In [1]:
# imports
import numpy as np
import pandas as pd

**Dataset n° 1 et n° 2**

The GeoNames geographical database covers all countries and contains over eleven million placenames that are available for download free of charge.

https://www.geonames.org/

#### Pays

Voir : https://www.geonames.org/countries/

In [ ]:
# dataset 1
var = pd.read_html('https://www.geonames.org/countries/',
                   header=0,
                   keep_default_na=False  # NA = North America
                  )
[x.shape for x in var]

In [ ]:
# pays
df_pays = var[1]
df_pays.head()

In [ ]:
# nombre de pays par continent
df_pays['Continent'].value_counts()

#### Villes

Voir : http://download.geonames.org/export/dump/

In [ ]:
# villes
df = pd.read_csv('cities500.zip',
                 sep='\t',
                 header=None,
                 keep_default_na=False,  # NA = North America
                 na_values=['', -9999],
                 names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 
                        'longitude', 'feature class', 'feature code', 'country code', 
                        'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 
                        'population', 'elevation', 'dem', 'timezone', 'modification date'],
                dtype={'admin1 code': str,
                       'admin2 code': str,
                       'admin3 code': str,
                       'admin4 code': str})
df.shape

In [ ]:
# villes
df.head()

In [ ]:
# nombre de villes par pays
df['country code'].value_counts().head(16)

**Exercice n° 1**

- Quelle est la ville la plus peuplée (population) ?
- Quelle est la ville la plus haute (elevation ou dem) ?
- Quelle est la ville la plus basse (elevation ou dem) ?
- La colonne "alternatenames" contient pour chaque ville les différents noms de chacune des villes, séparés par des virgules. Quelle ville possède le plus de noms différents ? Donnez la liste des noms.

### 1. Combiner les données

Union ensembliste :
- fonction `concat()` : concaténation de Series ou DataFrames à partir d'une liste
- méthode `append()` : concaténation de Series ou DataFrames à partir d'un objet Series ou DataFrame

`pd.concat([df1, df2]) <=> df1.append(df2)`

Jointure de bases de données :
- fonction `merge()` : jointure de 2 DataFrames ('on' ou bien 'left_on' + 'right_on', 'how', 'suffixes' pour les colonnes dupliquées)
- méthode `join()` : jointure d'un DataFrame à un autre ('on', 'how', 'rsuffix', 'lsuffix' pour les colonnes dupliquées)


`pd.merge(df1, df2) <=> df1.join(df2)`



Mot-clé 'how' :
- 'inner' (INNER JOIN) : intersection des valeurs des 2 colonnes
- 'left' (LEFT OUTER JOIN) : valeurs de la colonne de gauche
- 'right' (RIGHT OUTER JOIN) : valeurs de la colonne de droite
- 'outer' (OUTER JOIN) : union des valeurs des 2 colonnes

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [ ]:
# merge df et df_pays avec le code iso2
df = pd.merge(df,
              df_pays,
              left_on='country code',
              right_on='ISO-3166alpha2',
              how='left')
df.to_pickle('df_geo.pkl')
df.head()

In [ ]:
pd.read_pickle('df_geo.pkl')

In [ ]:
df.info()

In [ ]:
# vérification de l'identité des 2 colonnes (INUTILE !)
(df['country code'] == df['ISO-3166alpha2']).all()

In [ ]:
# nombre de villes par continent
df['Continent'].value_counts()

### 2. Stratégie split-apply-combine

La stratégie split-apply-combine consiste à :
- éclater les données en sous-groupes sur la base d'un critère (par ex., les valeurs d'une colonne)
- appliquer une fonction à chacun des sous-groupes indépendamment
- combiner les résultats en une structure de données

`df.groupby()` retourne un objet de type DataFrameGroupBy qui peut être vu comme un dictionnaire dont les clés sont les différentes valeurs de la colonne utilisée pour éclater les données, et dont les valeurs sont des sous-DataFrames ou des sous-Series correspondant aux données éclatées.

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

In [ ]:
# groupby Continent
cont_group = df.groupby('Continent')
type(cont_group)

In [ ]:
# ngroups
cont_group.ngroups

In [ ]:
# sous-groupe
cont_group.get_group('AN')

### Application de méthodes standard

- size() : group sizes
- count() : count of group
- mean() : mean of groups
- sum() : sum of group values
- std() : standard deviation of groups
- var() : variance of groups
- first() : first of group values
- last() : last of group values
- nth() : nth value
- min() : min of group values
- max() : max of group values

In [ ]:
# exemple avec size()
df.groupby('Continent').size()

In [ ]:
# exemple avec size()
df.groupby('Country').size()

In [ ]:
# exemple avec sum()
df.groupby('Continent').sum()

### Méthode aggregate() ou agg()

Applique une fonction, une liste de fonctions ou un dictionnaire de fonctions à un groupby.

#### Avec une fonction simple

In [ ]:
# exemple
df.groupby('Continent').agg('mean')  # idem que df.groupby('Continent').mean()

#### Avec une liste de fonctions

In [ ]:
# exemple
df.groupby('Continent').agg(['mean', 'std'])

#### Avec un dictionnaire de fonctions

In [ ]:
# exemple avec un dictionnaire
df.groupby('Continent').agg({'population': 'sum',
                             'elevation': 'mean',
                             'Country': ['min', 'max']})

### Méthode apply()

Applique une lambda ou une fonction définie à un groupby.

In [ ]:
df.info()

In [ ]:
# retourne la ville la plus peuplée d'un DataFrame
def top1city(group):
    return group.loc[group['population'].idxmax(), 'name']

In [ ]:
# avec df
top1city(df)

In [ ]:
# avec le groupby Continent
df.groupby('Continent').apply(top1city)

In [ ]:
# avec le groupby Country
df.groupby('Country').apply(top1city)

In [ ]:
# retourne les 3 villes les plus peuplées d'un DataFrame
def top3city(group):
    return group.nlargest(3, 'population')['name']

In [ ]:
# avec df
top3city(df)

In [ ]:
# par continent
df.groupby('Continent').apply(top3city)#.droplevel(1)  # supprime le multi-index

In [ ]:
cont_group.get_group('EU').nlargest(3, 'population')['name']

**Exercice n° 2**

Créez une fonction qui calcule la moyenne du nombre de noms alternatifs de chaque ville.

Testez-la sur tout le dataset.

Appliquez cette fonction sur un groupby 'feature code'. Quel code obtient la valeur la plus grande ?

Voir les features codes des pays : https://www.geonames.org/export/codes.html#P

### 3. Index hiérarchiques

pandas est capable de gérer des index hiérarchiques.

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

#### 3.1 Index

In [ ]:
# groupby 2 colonnes
# nombre de villes par pays et par continent
s = df.groupby(['Continent', 'Country']).size()
s

In [ ]:
# type de l'index
type(s.index)

In [ ]:
# niveaux de l'index : Continent, Country
s.index.levels

In [ ]:
# accès au premier niveau
s.loc['AF'] # ou s.loc[('AF',)]

In [ ]:
# accès au second niveau
s.loc[('AF', 'Algeria')]

In [ ]:
# reset Continent
s.reset_index('Continent') # ou s.reset_index(level=0)

In [ ]:
# reset Country
s.reset_index('Country') # ou s.reset_index(level=1)

In [ ]:
# swaplevel
s.swaplevel()

In [ ]:
# droplevel
s.droplevel(0)

#### 3.2 Columns

In [ ]:
# retour sur un dataframe avec des colonnes hiérarchiques
tab = df.groupby('Continent').agg({'population': 'sum',
                                   'elevation': 'mean',
                                   'Country': ['min', 'max']})
tab

In [ ]:
# columns
tab.columns

In [ ]:
# accès au premier niveau
tab.loc[:, 'Country']  # ou tab.loc[:, ('Country')]

In [ ]:
# accès au second niveau
tab.loc[:, ('Country', 'min')]

In [ ]:
# swaplevel
tab.swaplevel(axis=1)

In [ ]:
# droplevel
tab.droplevel(0, axis=1)

### 4. Méthodes de reshaping (2)

**pandas** possède plusieurs méthodes de reshaping qui généralisent les méthodes de pivot :
- `stack()` : move the inner-most (or the specified) column level to the inner-most index level
- `unstack()` : move the inner-most (or the specified) index level to the inner-most column level
- `swaplevel()` : swap 2 levels from index or from columns
- `droplevel()` : drop a level of an index or of a column

D'autres méthodes permettent de manipuler les index et peuvent être combinées :
- `set_index()` : move the specified column as the new index
- `reset_index()` : move the specified index as a new column
- `reindex()` : conform to a new index

Enfin, la méthode `melt()` permet de faire passer une table d'un format large vers un format long.

Toutes ces méthodes sont très utiles pour reformatter des data, notamment lorsqu'elles sont fournies dans un format pour les humains et doivent être transformées dans un format pour les machines.

Voir : https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

Voir également l'article Tidy Data.

In [ ]:
# exemple
s = df.groupby(['Continent', 'Country']).size()
s

In [ ]:
# exemple
s.unstack()

In [ ]:
# exemple
s.unstack('Continent')

In [ ]:
# exemple
tab = df.groupby('Continent').agg({'population': 'sum',
                                   'elevation': 'mean',
                                   'Country': ['min', 'max']})
tab

In [ ]:
# exemple
tab.stack()

In [ ]:
# exemple
tab.stack(0)

**Remarque** : la méthode `pivot_table()` peut être simulée avec un `groupby()` suivi d'un `unstack()`.

In [ ]:
# remarque pivot_table = groupby + unstack

df.pivot_table(values='Population', index='Country', columns='Continent', aggfunc='sum')

In [ ]:
# remarque pivot_table = groupby + unstack

df.groupby(['Country', 'Continent'])['Population'].sum().unstack()

Le `pivot_table()` est même un peu plus lent, mais plus facile à comprendre.

In [ ]:
# remarque pivot_table = groupby + unstack

%timeit df.pivot_table(values='Population', index='Country', columns='Continent', aggfunc='sum')

In [ ]:
# groupby + unstack
%timeit df.groupby(['Country', 'Continent'])['Population'].sum().unstack()

**Dataset n° 3**

United Nations (UNCTAD) with FDI inflows (Foreign direct investment), by region and economy from
1990 to 2018.

Ce fichier est en format large avec une colonne par année, facilement lisible par un humain.

In [3]:
# UNCTAB dataset
df_un = pd.read_excel('WIR19_tab01.xlsx',
                      header=2,
                      nrows=240,
                      engine='openpyxl')
df_un = df_un.drop(0)
df_un['Region/economy'] = df_un['Region/economy'].apply(str.strip)
df_un.columns = ['Region/economy'] + [int(col) for col in df_un.columns[1:]]
df_un

,Region/economy,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
1,World,204886.351424,153957.259850,162916.898975,220084.505159,254910.282247,341522.543094,388825.078858,480774.095268,690861.021060,...,1.172234e+06,1.365107e+06,1.561354e+06,1.470334e+06,1.431164e+06,1.357239e+06,2.033802e+06,1.918679e+06,1.497371e+06,1.297153e+06
2,Developed economies,170166.882709,114508.768046,107809.179346,141370.315360,150589.703875,219760.864472,236347.865354,285575.605654,508697.242961,...,6.487782e+05,6.790101e+05,8.168368e+05,7.417653e+05,6.948481e+05,6.230776e+05,1.268594e+06,1.197735e+06,7.592561e+05,5.568920e+05
3,Europe,102630.292871,81748.341024,74236.381665,79026.703380,88547.808451,135587.307684,130865.901072,155722.798345,298843.192160,...,4.334056e+05,4.098058e+05,4.775278e+05,4.257446e+05,3.511968e+05,2.833924e+05,7.150168e+05,6.116934e+05,3.840234e+05,1.718776e+05
4,European Union,95560.518749,78785.458885,74506.203225,78117.977606,82405.899729,130964.690115,124492.976251,144955.924290,285814.923823,...,3.867505e+05,3.626409e+05,4.347554e+05,3.764621e+05,3.450345e+05,2.656186e+05,6.358399e+05,5.561182e+05,3.405704e+05,2.776402e+05
5,Austria,653.000000,360.000000,1485.518380,1125.106168,2075.823025,1825.089897,4323.515257,2668.974480,4571.855469,...,9.268300e+03,2.575464e+03,1.061565e+04,3.988878e+03,5.719908e+03,4.577236e+03,1.487764e+03,-8.170196e+03,1.109211e+04,7.618219e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Tajikistan,0.000000,0.000000,9.000000,9.000000,12.000000,10.000000,18.000000,18.000000,29.940400,...,1.313200e+02,1.552784e+02,2.091933e+02,2.547928e+02,2.146462e+02,4.317106e+02,5.586260e+02,3.436861e+02,2.698773e+02,3.167354e+02
236,Turkmenistan,0.000000,0.000000,0.000010,79.000000,103.000000,233.000000,108.060000,107.860000,62.300000,...,4.553000e+03,3.632300e+03,3.391067e+03,3.129615e+03,2.861421e+03,3.830131e+03,3.042968e+03,2.243160e+03,2.085944e+03,1.985147e+03
237,Ukraine,0.000000,0.000000,200.000000,200.000000,159.000000,267.000000,521.000000,623.000000,743.000000,...,4.816000e+03,6.495000e+03,7.207000e+03,8.401000e+03,4.499000e+03,4.100000e+02,2.961000e+03,3.284000e+03,2.601000e+03,2.355000e+03
238,Uzbekistan,0.000000,0.000000,9.000000,48.000000,73.000000,-24.000000,90.000000,166.800000,139.600000,...,8.420000e+02,1.636449e+03,1.635145e+03,5.630405e+02,6.347013e+02,7.573580e+02,6.648900e+01,1.341390e+02,9.765300e+01,4.124090e+02


On peut utiliser `stack()` + `reset_index()` pour le passer en format long, dit normalisé.

In [4]:
# pour le passer au format long
# on peut utiliser stack + reset_index
df_un.set_index('Region/economy').stack().reset_index()

,Region/economy,level_1,0
0,World,1990,204886.351424
1,World,1991,153957.259850
2,World,1992,162916.898975
3,World,1993,220084.505159
4,World,1994,254910.282247
...,...,...,...
6926,Georgia,2014,1817.981000
6927,Georgia,2015,1652.507000
6928,Georgia,2016,1565.838000
6929,Georgia,2017,1894.491000


In [5]:
# vérification de la longueur
len(df_un) * (df_un.shape[1] - 1)

6931

La méthode `melt()` fait aussi ça très bien.

Elle prend comme arguments :
- le DataFrame,
- la liste des colonnes associées à l'identité des enregistrements
- la liste des colonnes associées aux différentes valeurs considérées

Elle génère un DataFrame normalisé avec les colonnes associées à l'identité des enregistrements, ainsi qu'une colonne "variable" correspondant aux noms des anciennes colonnes de valeurs et une colonne "value" correspondant aux valeurs des anciennes colonnes de valeurs.

In [6]:
# ou bien tout simplement
# la méthode melt
tab = df_un.melt(id_vars=['Region/economy'],
                 value_vars=list(df_un.columns[1:]))
tab

,Region/economy,variable,value
0,World,1990,204886.351424
1,Developed economies,1990,170166.882709
2,Europe,1990,102630.292871
3,European Union,1990,95560.518749
4,Austria,1990,653.000000
...,...,...,...
6926,Tajikistan,2018,316.735371
6927,Turkmenistan,2018,1985.147000
6928,Ukraine,2018,2355.000000
6929,Uzbekistan,2018,412.409000


In [7]:
# pour revenir pratiqument au DataFrame de départ
tab.set_index(['Region/economy', 'variable']).unstack().reset_index()

Region/economy          value                                \
variable                                1990           1991           1992   
0                 Afghanistan       0.000010      -0.280000       0.360000   
1                      Africa    2845.173268    3543.551296    3839.975371   
2                     Albania       0.000000       0.000000      20.000000   
3                     Algeria      40.000000      80.000000      30.000000   
4                      Angola    -334.500000     664.100000     288.000000   
..                        ...            ...            ...            ...   
234                     World  204886.351424  153957.259850  162916.898975   
235                     Yemen    -130.900000     282.500000     718.000000   
236       Yugoslavia (former)      67.000000     119.000000       0.000000   
237                    Zambia     202.780000      34.340000      45.000000   
238                  Zimbabwe      12.200000       2.800000      19.000000   

                                                                      \
variable           1993           1994           1995           1996   
0             -0.020000       0.020000      -0.093327       0.690000   
1           5443.865367    6104.516476    5665.131718    6047.850348   
2             68.000000      52.970000      70.050000      90.090000   
3              0.001000       0.001000       0.001000     270.000000   
4            302.000000     170.100000     472.000000     180.630000   
..                  ...            ...            ...            ...   
234       220084.505159  254910.282247  341522.543094  388825.078858   
235          903.000000      15.800000    -217.700000     -60.100000   
236            0.000000       0.000000       0.000000       0.000000   
237          314.400000      40.000000     107.000000     127.100000   
238           38.000000      41.000000     117.700000      80.900000   

                                        ...                              \
variable           1997           1998  ...          2009          2010   
0             -1.460000      -0.010000  ...  1.975130e+02  5.420100e+01   
1          11030.171944    9991.920512  ...  5.665231e+04  4.662015e+04   
2             47.540000      45.010797  ...  9.959297e+02  1.050708e+03   
3            260.000000     606.600000  ...  2.753760e+03  2.301230e+03   
4            411.650000    1114.000000  ...  2.205298e+03 -3.227211e+03   
..                  ...            ...  ...           ...           ...   
234       480774.095268  690861.021060  ...  1.172234e+06  1.365107e+06   
235         -138.500000    -219.400000  ...  1.291940e+02  1.886280e+02   
236            0.000000       0.000000  ...  0.000000e+00  0.000000e+00   
237          217.000000     238.000000  ...  6.948000e+02  1.729300e+03   
238          135.100000     444.300000  ...  1.050000e+02  1.659000e+02   

                                                                  \
variable          2011          2012          2013          2014   
0         5.762100e+01  4.722700e+01  3.763900e+01  4.351000e+01   
1         4.563344e+04  5.685371e+04  5.007484e+04  5.390601e+04   
2         8.762683e+02  8.554402e+02  1.265550e+03  1.109960e+03   
3         2.580350e+03  1.499450e+03  1.696867e+03  1.506730e+03   
4        -3.023771e+03 -1.464628e+03 -7.120017e+03  3.657515e+03   
..                 ...           ...           ...           ...   
234       1.561354e+06  1.470334e+06  1.431164e+06  1.357239e+06   
235      -5.184203e+02 -5.310000e+02 -1.336000e+02 -2.331050e+02   
236       0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
237       1.108500e+03  1.731500e+03  2.099800e+03  1.488700e+03   
238       3.870000e+02  3.995000e+02  4.000000e+02  5.448000e+02   

                                                                  
variable          2015          2016          2017          2018  
0         1.631030e+02  9.359000e+01  5.339000e+01  1.392000e+02  
1

**Exercice n° 3**

Faites la jointure de `df_un` avec `df_pays` en utilisant la fonction `merge()` ou la méthode `join()`.

Comparez le nombre de lignes entre le résultat et `df_pays`. D'où vient le problème ?

Faire le mapping avec un dictionnaire de transcodification des noms des pays (voir le fichier `mapping.py`).

Repassez en format long avec la méthode `melt()`.